<center><h1>Kaggle Competition: Titanic Data Set</h1></center>

This Jupyter Notebook represents my attempt at competing in the Kaggle competition for the Titanic Data Set.  I'll add in notes and explanation all throughout in order to clarify my approach and my thoughts at each step.  



<center><h3>Step 1: Data Wrangling</h3></center>

In this step, I'll import the data from the CSV files, explore the data, and "clean" it by putting it in a format more conducive to machine learning.  

In [10]:
# Import all necessary libraries
import pandas as pd
import time
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [11]:
raw_df = pd.read_csv("train.csv")
pred_df = pd.read_csv("test.csv")

# Separate out the labels into a separate data frame.  
train = raw_df.drop("Survived", axis=1, inplace=False)
labels = raw_df["Survived"]

# Replace categorical values for embarked, sex with numerical values
train["Embarked"] = train["Embarked"].map({"S": 0, "C": 1, "Q": 2})
train["Sex"] = train["Sex"].map({"female": 0, "male": 1})

# Drop names, ticket numbers, cabins
train.drop("Name", axis=1, inplace=True)
train.drop("Ticket", axis=1, inplace=True)
train.drop("Cabin", axis=1, inplace=True)
train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.0,1,0,7.2500,0.0
1,2,1,0,38.0,1,0,71.2833,1.0
2,3,3,0,26.0,0,0,7.9250,0.0
3,4,1,0,35.0,1,0,53.1000,0.0
4,5,3,1,35.0,0,0,8.0500,0.0


The data is in better shape, but we still have to deal with our null values.  

In [12]:
# Deal with Null Values
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].median())

In [13]:
# Store column names for data that will be used by our classifier.
parameters = ["Pclass", "Sex", "Age", "SipSp", "Parch", "Fare", "Embarked"]

In [14]:
train["Age"].describe()

count    891.000000
mean      29.361582
std       13.019697
min        0.420000
25%       22.000000
50%       28.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [19]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred)


def train_predict(clf, X_train, y_train, X_test, y_test):
    print("-"*50)
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print("-"*50)
    
def make_predictions(clf, X_test):
    start = time.time()
    pred = clf.predict(X_test)
    end = time.time()
    
    return pred

In [16]:
scorer = make_scorer(f1_score)
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=.30, random_state=42)

# We'll try several different Ml algorithms, and go with the one that has the highest F1 Score.  
clf1 = DecisionTreeClassifier()
clf2 = SVC()
clf3 = GaussianNB()
clf4 = RandomForestClassifier()
clf5 = XGBClassifier(
        nthread = 4,
        silent = 1
        )

all_clfs = [clf1, clf2, clf3, clf4, clf5]

for clf in all_clfs:
    train_predict(clf, X_train, y_train, X_test, y_test)



--------------------------------------------------
Training a DecisionTreeClassifier using a training set size of 623. . .
Trained model in 0.0029 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7143.
--------------------------------------------------
--------------------------------------------------
Training a SVC using a training set size of 623. . .
Trained model in 0.0192 seconds
Made predictions in 0.0105 seconds.
F1 score for training set: 0.9935.
Made predictions in 0.0044 seconds.
F1 score for test set: 0.0526.
--------------------------------------------------
--------------------------------------------------
Training a GaussianNB using a training set size of 623. . .
Trained model in 0.0012 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.7188.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7545.
--------------------------------------------------

Initial Results:

SVC was horrible.  There are several reasons that this could be.

Best untuned results: Gaussian Naive Bayes and XGBoost Classifiers.  There's not enough of a difference between the scores for reach to declare one a clear winner.  In that case, we'll tune them both and see which one works best.  

There's not much tuning that can be done to A GaussianNB Classifier--the only parameter that you can tweak is the priors, and that's something better calculated from the data itself.  

That leaves XGBoost.  

In [17]:
params = {'objective':['binary:logistic'],
              'learning_rate': [ii * 0.01 for ii in range(1, 6)], #so called `eta` value
              'max_depth': [2, 3, 4],
              'min_child_weight': [ii for ii in range(1, 5)],
              'silent': [0],
              'subsample': [0.7, 0.8, 0.9],
              'colsample_bytree': [0.5, 0.6, 0.7, 0.8],
              'n_estimators': [5, 10, 15], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [42]}

clf = GridSearchCV(clf5, parameters, 
                   scoring='f1',
                    refit=True)
clf.fit(X_train, y_train)
clf = clf.best_estimator_



In [18]:
train_predict(clf, X_train, y_train, X_test, y_test)


--------------------------------------------------
Training a XGBClassifier using a training set size of 623. . .
Trained model in 0.0082 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.7820.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.7263.
--------------------------------------------------


In [32]:
test = pd.read_csv("test.csv")
test["Embarked"] = test["Embarked"].map({"S": 0, "C": 1, "Q": 2})
test["Sex"] = test["Sex"].map({"female": 0, "male": 1})

# Drop names, ticket numbers, cabins
test.drop("Name", axis=1, inplace=True)
test.drop("Ticket", axis=1, inplace=True)
test.drop("Cabin", axis=1, inplace=True)
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Embarked"] = test["Embarked"].fillna(test["Embarked"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [43]:
pred = pd.DataFrame(make_predictions(clf, test))
pred["PassengerId"] = test["PassengerId"]

cols = pred.columns.tolist()
cols = [cols[1], cols[0]]
pred = pred[cols]
pred.to_csv("output.csv", index=False)
pred

,PassengerId,0
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,0
9,901,0
